## Load your data

In [1]:
# import packages
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import cnmfereview as cr
import config as cfg
import os
from joblib import dump, load

MODELDIR = Path('../best_models')

data = cr.Dataset(
    data_paths=cfg.data_paths,
    exp_id=cfg.exp_id,
    img_shape=cfg.img_shape,
    img_crop_size=cfg.img_crop_size,
    max_trace=cfg.max_trace_len,
)

x_train, x_test, y_train, y_test = data.split_training_test_data(
    test_split=.20,
    seed=10
)
x_train.shape

No preprocessing on spatial data
File ../data/cr_tutorialA_cropped.npy already exists and has been loaded instead.
No preprocessing on trace data.                   ../data/cr_tutorialCraw_normalized.npy already                   exists and has been loaded instead.
Successfully loaded data.
Training and test data loaded


(11603, 6900)

_________________
**NOTE: Remove the next cell when training your own models.** This step uses fewer ROIs (only ~3000 instead of 11 000) in the tutorial dataset to speed up computation in the tutorial. Do not do this when you are training your own data. You want to use as many data samples as possible to get the best results in practice. 

In [2]:
# remove or comment out this cell when using on your own data
# from sklearn.model_selection import train_test_split
# x_train, _, y_train, _ = train_test_split(x_train, y_train, test_size=0.75)

print(f"Number of samples in training set: {x_train.shape[0]}") 
print(f"Number of samples in test set: {x_test.shape[0]}")

Number of samples in training set: 11603
Number of samples in test set: 2901


# Train the saved models on your data
### Deep Classifier

In [3]:
import ignite
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.handlers import ModelCheckpoint, EarlyStopping
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tensorboardX import SummaryWriter
from nn.model import Model
import optuna

In [4]:
data.spatial.shape, data.trace.shape, data.targets.shape
x_train, x_test, y_train, y_test = data.split_training_test_data(
    test_split=.20, seed=10, for_deep=True)

class datasets(torch.utils.data.Dataset):
    def __init__(self, x, y, device):
        self.x, self.y = x, y
        self.device = device
    def __len__(self):
        return self.y.shape[0]
    def __getitem__(self, i):
        data = (self.x[0][i].to(self.device), self.x[1][i].to(self.device))
        return data, self.y[i].to(self.device)

device = 'cuda:0'
trainsets = datasets(x_train, y_train, device)
testsets = datasets(x_test, y_test, device)
train_loader = torch.utils.data.DataLoader(trainsets, batch_size=32)
test_loader = torch.utils.data.DataLoader(testsets, batch_size=32)

Training and test data loaded


In [5]:
def train(model):
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    criterion = F.binary_cross_entropy
    epochs = 30

    def preprocess(y):
        return torch.round(y[0]), y[1]

    precision = ignite.metrics.Precision(preprocess, average=False)
    recall = ignite.metrics.Recall(preprocess, average=False)
    F1 = (precision * recall * 2 / (precision + recall)).mean()

    trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
    evaluator = create_supervised_evaluator(
        model,
        metrics={'accuracy': ignite.metrics.Accuracy(preprocess),
                 'f1': F1,
                 'cross_entropy': ignite.metrics.Loss(criterion)},
        device=device)
    writer = SummaryWriter()

    @trainer.on(Events.ITERATION_COMPLETED)
    def log_training_loss(engine):
        i = (engine.state.iteration - 1) % len(train_loader) + 1
        if i % 500 == 0:
            print(f"Epoch[{engine.state.epoch}] Iteration[{i}/{len(train_loader)}] "
                  f"Loss: {engine.state.output:.2f}")
            writer.add_scalar("training/loss", engine.state.output, engine.state.iteration)

    def write_metrics(metrics, writer, mode: str, epoch: int):
        """print metrics & write metrics to log"""
        avg_accuracy = metrics['accuracy']
        avg_nll = metrics['cross_entropy']
        avg_f1 = metrics['f1']
        print(f"{mode} Results - Epoch: {epoch}  "
              f"Avg accuracy: {avg_accuracy:.2f} Avg loss: {avg_nll:.2f} "
              f"Avg F1: {avg_f1:.2f}")
        writer.add_scalar(f"{mode}/avg_loss", avg_nll, epoch)
        writer.add_scalar(f"{mode}/avg_accuracy", avg_accuracy, epoch)

    @trainer.on(Events.EPOCH_COMPLETED)
    def log_training_results(engine):
        evaluator.run(train_loader)
        metrics = evaluator.state.metrics
        write_metrics(metrics, writer, 'training', engine.state.epoch)

    @trainer.on(Events.EPOCH_COMPLETED)
    def log_validation_results(engine):
        evaluator.run(test_loader)
        metrics = evaluator.state.metrics
        write_metrics(metrics, writer, 'validation', engine.state.epoch)

    handler = ModelCheckpoint(dirname='./checkpoints', filename_prefix='sample',
                              n_saved=5, create_dir=True, require_empty=False)
    trainer.add_event_handler(Events.EPOCH_COMPLETED, handler, {'mymodel': model})

    handler = EarlyStopping(
        patience=5,
        score_function=lambda x: x.state.metrics['f1'],
        trainer=trainer)
    evaluator.add_event_handler(Events.COMPLETED, handler)

    trainer.run(train_loader, max_epochs=epochs)
    return handler.best_score

In [14]:
# 5 is best
# scores = []
# for i in range(1, 6):
#     model = Model(
#         s_stage='ResNet',
#         res_block_num=i,
#     )
#     scores.append(train(model.to(device)))
for i in range(5):
    print(i+1, scores[i])

1 0.9178238411484538
2 0.9309892416688533
3 0.9255801547079221
4 0.9201073087744553
5 0.9367221276038256


In [ ]:
def optimaze_san(trial):
    block_num = trial.suggest_int('block_num', 1, 5)
    layer_size_hop = trial.suggest_int('layer_size_hop', 2, 5)
    kernel_size = trial.suggest_int('kernel_size', 3, 7, 2)

    layers = [3]
    kernels = [3]
    for i in range(1, block_num):
        layers.append(2 + i*layer_size_hop)
        kernels.append(kernel_size)
    
    model = Model(
        s_stage='SAN',
        san_layers=layers,
        san_kernels=kernels,
    )
    score = train(model.to(device))
    return -score
study = optuna.create_study()
study.optimize(optimaze_san, n_trials=30)

[I 2020-09-29 01:23:31,403] A new study created in memory with name: no-name-91a4e327-ac95-47f0-beca-c62960800e7e


spatial feature len: 12800, temporal feature len: 300
training Results - Epoch: 1  Avg accuracy: 0.88 Avg loss: 0.31 Avg F1: 0.90
validation Results - Epoch: 1  Avg accuracy: 0.87 Avg loss: 0.30 Avg F1: 0.89
training Results - Epoch: 2  Avg accuracy: 0.89 Avg loss: 0.27 Avg F1: 0.91
validation Results - Epoch: 2  Avg accuracy: 0.89 Avg loss: 0.27 Avg F1: 0.92
training Results - Epoch: 3  Avg accuracy: 0.89 Avg loss: 0.26 Avg F1: 0.92
validation Results - Epoch: 3  Avg accuracy: 0.89 Avg loss: 0.27 Avg F1: 0.92
training Results - Epoch: 4  Avg accuracy: 0.89 Avg loss: 0.25 Avg F1: 0.92
validation Results - Epoch: 4  Avg accuracy: 0.90 Avg loss: 0.26 Avg F1: 0.92
training Results - Epoch: 5  Avg accuracy: 0.89 Avg loss: 0.25 Avg F1: 0.92
validation Results - Epoch: 5  Avg accuracy: 0.89 Avg loss: 0.26 Avg F1: 0.92
training Results - Epoch: 6  Avg accuracy: 0.90 Avg loss: 0.24 Avg F1: 0.92
validation Results - Epoch: 6  Avg accuracy: 0.90 Avg loss: 0.25 Avg F1: 0.92
training Results - Epo

[I 2020-09-29 01:42:44,537] Trial 0 finished with value: -0.9284347998153156 and parameters: {'block_num': 3, 'layer_size_hop': 5, 'kernel_size': 3}. Best is trial 0 with value: -0.9284347998153156.


spatial feature len: 51200, temporal feature len: 300
training Results - Epoch: 1  Avg accuracy: 0.83 Avg loss: 0.41 Avg F1: 0.88
validation Results - Epoch: 1  Avg accuracy: 0.84 Avg loss: 0.39 Avg F1: 0.89
training Results - Epoch: 2  Avg accuracy: 0.88 Avg loss: 0.35 Avg F1: 0.91
validation Results - Epoch: 2  Avg accuracy: 0.88 Avg loss: 0.34 Avg F1: 0.91
training Results - Epoch: 3  Avg accuracy: 0.89 Avg loss: 0.31 Avg F1: 0.91
validation Results - Epoch: 3  Avg accuracy: 0.89 Avg loss: 0.30 Avg F1: 0.91
training Results - Epoch: 4  Avg accuracy: 0.86 Avg loss: 0.36 Avg F1: 0.90
validation Results - Epoch: 4  Avg accuracy: 0.87 Avg loss: 0.34 Avg F1: 0.91
training Results - Epoch: 5  Avg accuracy: 0.89 Avg loss: 0.31 Avg F1: 0.91
validation Results - Epoch: 5  Avg accuracy: 0.88 Avg loss: 0.34 Avg F1: 0.91


[I 2020-09-29 01:44:15,296] Trial 1 finished with value: -0.914625288874216 and parameters: {'block_num': 1, 'layer_size_hop': 5, 'kernel_size': 3}. Best is trial 0 with value: -0.9284347998153156.


spatial feature len: 6400, temporal feature len: 300
training Results - Epoch: 1  Avg accuracy: 0.88 Avg loss: 0.30 Avg F1: 0.91
validation Results - Epoch: 1  Avg accuracy: 0.89 Avg loss: 0.31 Avg F1: 0.91
training Results - Epoch: 2  Avg accuracy: 0.89 Avg loss: 0.28 Avg F1: 0.91
validation Results - Epoch: 2  Avg accuracy: 0.89 Avg loss: 0.28 Avg F1: 0.92
training Results - Epoch: 3  Avg accuracy: 0.88 Avg loss: 0.27 Avg F1: 0.91
validation Results - Epoch: 3  Avg accuracy: 0.89 Avg loss: 0.28 Avg F1: 0.92
training Results - Epoch: 4  Avg accuracy: 0.88 Avg loss: 0.27 Avg F1: 0.91
validation Results - Epoch: 4  Avg accuracy: 0.88 Avg loss: 0.27 Avg F1: 0.91
training Results - Epoch: 5  Avg accuracy: 0.89 Avg loss: 0.26 Avg F1: 0.92
validation Results - Epoch: 5  Avg accuracy: 0.89 Avg loss: 0.26 Avg F1: 0.92
training Results - Epoch: 6  Avg accuracy: 0.89 Avg loss: 0.26 Avg F1: 0.91
validation Results - Epoch: 6  Avg accuracy: 0.89 Avg loss: 0.27 Avg F1: 0.92
training Results - Epoc

[I 2020-09-29 02:06:55,306] Trial 2 finished with value: -0.9212089899250839 and parameters: {'block_num': 4, 'layer_size_hop': 5, 'kernel_size': 7}. Best is trial 0 with value: -0.9284347998153156.


spatial feature len: 6400, temporal feature len: 300


In [ ]:
def optimaze_lstm(trial):
    model = Model(
        s_stage='ResNet',
        res_block_num=4,
        t_hidden_dim=trial.suggest_int('t_hidden_dim', 50, 500, 50),
        t_output_dim=trial.suggest_int('t_output_dim', 50, 500, 50),
    )
    score = train(model.to(device))
    return -score
study = optuna.create_study()
study.optimize(optimaze_lstm, n_trials=30)

### TPOT Classifier

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, accuracy_score
from sklearn.svm import LinearSVC

# this was the final TPOT exported pipeline that acheived the highest F1 score
tpot_model = LinearSVC(C=0.1, dual=False, loss="squared_hinge", penalty="l1", tol=0.1)
tpot_model.fit(x_train, y_train)

In [ ]:
tpot_y_pred = tpot_model.predict(x_train)
print("Accuracy:", accuracy_score(y_train, tpot_y_pred))
print("f1:", f1_score(y_train, tpot_y_pred))
dump(tpot_model, MODELDIR / f'{cfg.exp_id}_tpot.joblib') 

Save a copy of the model finetuned on your data to use again in the future to predict without having to retrain.
### AutoSklearn Classifier

In [ ]:
import autosklearn
import sklearn
# load the AutoSklearn ensemble object
askl = load(MODELDIR / 'cr_tutorial_askl.joblib')
askl.refit(x_train, y_train)

In [ ]:
results_automl = askl.predict(x_test)
print("Accuracy:", accuracy_score(y_test, results_automl))
print("f1:", f1_score(y_test, results_automl))
dump(askl, MODELDIR / f'{cfg.exp_id}_askl.joblib')

# Apply classifiers to unlabeled data

In [ ]:
askl = load(MODELDIR / f'{cfg.exp_id}_askl.joblib');
tpot_model = load(MODELDIR / f'{cfg.exp_id}_tpot.joblib')
cfg.img_shape

In [ ]:
unseen_data = cr.UnlabeledDataset(
    mat_file='../data/unlabeled_rois_DM298.mat',
    img_shape={'x': 284, 'y': 231},
    img_crop_size=cfg.img_crop_size,
    max_trace=cfg.max_trace_len)

In [ ]:
cfg.img_shape, cfg.img_crop_size, cfg.max_trace_len

In [ ]:
pred_askl = askl.predict(unseen_data.combined)
pred_tpot = tpot_model.predict(unseen_data.combined)

In [ ]:
# preview the ROIs labeled by askl as "positives"
positive_askl = np.where(pred_askl==1)[0]
# limit to only show 10 at once, you can play around with this of course
cr.plot_rois(unseen_data, positive_askl[:10])

In [ ]:
# preview the ROIs labeled by askl as "negatives"
negative_askl = np.where(pred_askl==0)[0]
# limit to only show 10 at once, you can play around with this of course
cr.plot_rois(unseen_data, negative_askl[:10])

In [ ]:
gt_label = [1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
accuracy_score(gt_label, pred_askl), f1_score(gt_label, pred_askl)

In [ ]:
# preview the ROIs labeled by TPOT as "negatives"
cr.plot_rois(unseen_data, np.where(pred_tpot==0)[0][:10])

# Apply reviews

In [ ]:
unseen_data.apply_labels(pred_askl)

In [ ]:
# load the file to check the results
from scipy.io import loadmat, savemat

labeled_data = loadmat('../data/unlabeled_rois_automl.mat')